In [1]:
pip install selenium pandas webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import re

# Setup Selenium WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Open LinkedIn Jobs Page with filters for India and remote jobs
job_roles = ["Application Support Engineer", "IT Support Engineer", "Site Reliability Engineer"]
location = "India"
remote = "true"

job_list = []

for role in job_roles:
    search_url = f"https://www.linkedin.com/jobs/search/?keywords={role.replace(' ', '%20')}&location={location}&f_WT={remote}"
    driver.get(search_url)
    time.sleep(5)  # Wait for page to load

    # Scroll to load more jobs
    for _ in range(3):  # Adjust range to load more results
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        time.sleep(3)

    # Extract job details
    jobs = driver.find_elements(By.CLASS_NAME, "base-search-card")

    for job in jobs:
        try:
            title = job.find_element(By.CLASS_NAME, "base-search-card__title").text
            company = job.find_element(By.CLASS_NAME, "base-search-card__subtitle").text
            location = job.find_element(By.CLASS_NAME, "job-search-card__location").text
            link = job.find_element(By.TAG_NAME, "a").get_attribute("href")
            
            job_list.append({
                "Title": title,
                "Company": company,
                "Location": location,
                "Link": link,
                "Email": ""  # Placeholder for HR email
            })
        except Exception as e:
            print("Skipping job due to error:", e)

# Extract emails from job descriptions
for job in job_list:
    try:
        driver.get(job["Link"])
        time.sleep(5)  # Wait for page to load
        
        # Extract page text
        page_text = driver.page_source
        
        # Find email addresses
        emails = re.findall(r'[\w\.-]+@[\w\.-]+', page_text)
        
        if emails:
            job["Email"] = emails[0]  # Save first found email
    except Exception as e:
        print("Could not extract email for", job["Title"], "due to error:", e)

# Save to CSV
job_df = pd.DataFrame(job_list)
job_df.to_csv("linkedin_jobs_with_emails.csv", index=False)
job_df.to_csv("/Users/applicate/Downloads/linkedin_jobs_with_emails.csv", index=False)

print("Scraping complete! Data saved to linkedin_jobs_with_emails.csv")

driver.quit()

Scraping complete! Data saved to linkedin_jobs_with_emails.csv


In [9]:
import smtplib
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import time
import os

# Load scraped job data
job_df = pd.read_csv("linkedin_jobs_with_emails.csv")

# Your Gmail credentials (Enable "Less secure apps" access if needed)
EMAIL_ADDRESS = "chetansethi58@gmail.com"
EMAIL_PASSWORD = "amwh tjyg kmun ajpr"
RESUME_PATH = "/Users/applicate/Downloads/ChetanSethi_Resume.pdf"

# Extract actual filename from path
resume_filename = os.path.basename(RESUME_PATH)

# Email content
def create_email_body(name, company):
    return f"""
    Dear {name},

    I hope you're doing well. I came across a job opening at {company} and wanted to express my interest in the role.

    I have experience in Technical/Application Support and would love to contribute my skills to your team. 
    Please find my resume attached for consideration.

    Looking forward to hearing from you.

    Thanks and Regards,
    Chetan Sethi
    """

# Setup SMTP server
server = smtplib.SMTP("smtp.gmail.com", 587)
server.starttls()
server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)

# Send emails
for index, row in job_df.iterrows():
    if pd.notna(row["Email"]):
        recipient_email = row["Email"]
        company = row["Company"]
        name = "Hiring Manager"  # Adjust if recipient names are available

        msg = MIMEMultipart()
        msg['From'] = EMAIL_ADDRESS
        msg['To'] = recipient_email
        msg['Subject'] = "Application for Support Engineer Role"
        
        body = create_email_body(name, company)
        msg.attach(MIMEText(body, 'plain'))
        
        # Attach resume
        attachment = open(RESUME_PATH, "rb")
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())
        encoders.encode_base64(part)
        part.add_header("Content-Disposition", f"attachment; filename= resume.pdf")
        msg.attach(part)
        attachment.close()
        
        # Send email
        try:
            server.sendmail(EMAIL_ADDRESS, recipient_email, msg.as_string())
            print(f"Email sent to {recipient_email}")
        except Exception as e:
            print(f"Failed to send email to {recipient_email}: {e}")
        
        time.sleep(5)  # Delay to avoid spam detection

# Use threading to send emails faster
with ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(send_email, row["Email"], row["Company"]) for _, row in job_df.iterrows() if pd.notna(row["Email"])]

# Close server connection
server.quit()

print("All emails sent!")

Failed to send email to talent@xplortechnologies.com.: {'talent@xplortechnologies.com.': (553, b'5.1.3 The recipient address <talent@xplortechnologies.com.> is not a valid\n5.1.3 RFC 5321 address. For more information, go to\n5.1.3  https://support.google.com/a/answer/3221692 and review RFC 5321\n5.1.3 specifications. d9443c01a7336-21de3308e32sm32070185ad.207 - gsmtp')}
Email sent to recruitment@redwood.com
Email sent to Engineer_Tanvi_HCL@1
Email sent to Engineer_Tanvi_HCL@1
Email sent to Engineer_Tanvi_HCL@1
Failed to send email to careers@flexera.com.: {'careers@flexera.com.': (553, b'5.1.3 The recipient address <careers@flexera.com.> is not a valid RFC 5321\n5.1.3 address. For more information, go to\n5.1.3  https://support.google.com/a/answer/3221692 and review RFC 5321\n5.1.3 specifications. d9443c01a7336-21de3308e32sm32070185ad.207 - gsmtp')}
Email sent to Engineer_Tanvi_HCL@1
Email sent to Engineer_Tanvi_HCL@3
Email sent to recruiting-lp@liveperson.com
Email sent to Engineer_Ta

NameError: name 'ThreadPoolExecutor' is not defined